In [2]:
# imports
import pandas as pd
import numpy as np
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [29]:
# Load data on our stations from Part 1
berlin_bikes = pd.read_csv('../data/berlin_bikes.csv')

# Whoops, it exported with an extra column that just holds the index. Let's drop it.
berlin_bikes = berlin_bikes.drop('Unnamed: 0',axis=1)

# In Foursquare, lat & long params need to be passed as '<lat>,<long>'
# Create a column of strings formatted as such so we don't have to do this nasty conversion when passing 'll' in params
berlin_bikes['lat_long'] = berlin_bikes['lat'].astype(str) + ',' + berlin_bikes['long'].astype(str)

# Drop the old redundant columns that are cluttering our simple dataframe.
berlin_bikes = berlin_bikes.drop(['lat','long'],axis=1)
print(berlin_bikes.head(3))


# [ Our station data is now set up appropriately ]

   bikes_available  bikes_in_use  total_bikes             lat_long
0                4             0            4  52.504157,13.335328
1                0             4            4   52.496986,13.29121
2                3             2            5  52.498323,13.296157


In [41]:
# Set out API key for Foursquare
api_key = os.environ["FOURSQUARE_API_KEY"]

# Set Foursquare url
url = 'https://api.foursquare.com/v3/places/search'

# Function that sends a request with each passed location
def general_1km_scan(location):
    # Request .Json response and attatch our key.
    headers = {
        'accept': 'application/json',
        'Authorization': api_key
    }
    # only needed parameter is lat and long
    parameters = {
        'll': location
    }

    response = requests.get(url,headers=headers,params=parameters)

    return response.json()


# [ This for loop calls the above function for every station in city ]
'''
for station in berlin_bikes['lat_long'].values:
    print(general_1km_scan(station))
'''
# There are 1808 stations in Berlin so I'm going to run a variation of the loop that takes the first 3


for station in berlin_bikes['lat_long'][:3].values:
    print(general_1km_scan(station))
# That's a ton of data

{'results': [{'fsq_id': '528e06ab11d2a05174deff6a', 'categories': [{'id': 10024, 'name': 'Movie Theater', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_', 'suffix': '.png'}}], 'chains': [], 'distance': 235, 'geocodes': {'main': {'latitude': 52.505882, 'longitude': 13.334317}, 'roof': {'latitude': 52.505882, 'longitude': 13.334317}}, 'link': '/v3/places/528e06ab11d2a05174deff6a', 'location': {'address': 'Hardenbergstraße 29A', 'country': 'DE', 'cross_street': '', 'formatted_address': 'Hardenbergstraße 29A, 10623 Berlin', 'locality': 'Berlin', 'postcode': '10623', 'region': 'Berlin'}, 'name': 'Zoo Palast', 'related_places': {'children': [{'fsq_id': '55de1309498ee36e5537b744', 'name': 'Zoo Loge Premium Lounge'}, {'fsq_id': '52a32e66498e76d72b69b85b', 'name': 'Zooba'}]}, 'timezone': 'Europe/Berlin'}, {'fsq_id': '5bc635e9d48ec1002cc10677', 'categories': [{'id': 13031, 'name': 'Burger Joint', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [45]:
# Trim the fat on this response by using the fields parameter
def general_1km_scan(location):
    headers = {
        'accept': 'application/json',
        'Authorization': api_key
    }
    # Add 'fields' to parameters wtih what we want returned.
    parameters = {
        'll': location,
        'fields' : 'name,distance,categories'
    }

    response = requests.get(url,headers=headers,params=parameters)

    return response.json()
general_1km_scan(berlin_bikes['lat_long'][0])

{'results': [{'categories': [{'id': 10024,
     'name': 'Movie Theater',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
      'suffix': '.png'}}],
   'distance': 235,
   'name': 'Zoo Palast'},
  {'categories': [{'id': 13031,
     'name': 'Burger Joint',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
      'suffix': '.png'}}],
   'distance': 310,
   'name': 'Burgermeister GmbH'},
  {'categories': [{'id': 10024,
     'name': 'Movie Theater',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
      'suffix': '.png'}}],
   'distance': 374,
   'name': 'Astor Film Lounge'},
  {'categories': [{'id': 10028,
     'name': 'Art Museum',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
      'suffix': '.png'}}],
   'distance': 423,
   'name': 'C/O Berlin'},
  {'categories': [{'id': 13310,
     'name': 'Modern European Re

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating